INTRODUCTION


This project will analyze neighborhoods between Toronto, Canada and New York City, United States. My family and I want to move abroad and we are considering these two options. We want insight into the neighborhoods and local businesses in the cities so that we may have the optimum living standards and quality of life. This project will explore the similarities and dissimilarities between certain neighborhoods in the two cities, and determine which neighborhoods best fit our needs. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!pip install lxml
import lxml
print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Download Toronto Dataset

In [2]:
#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# transform the data

In [3]:
#Remove Boroughs that are 'Not assigned'
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']
df_toronto.rename(columns = {"Neighbourhood": "Neighborhood"}, inplace = True)

#remove duplicates
df_toronto = df_toronto.drop_duplicates()
#Set index
df_toronto = df_toronto.set_index('Postal Code')
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto['Neighborhood'].replace("Not assigned", df_toronto["Borough"],inplace=True)

df_toronto.shape

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(103, 2)

In [4]:
df_toronto.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Join neighborhood table with latitude and longitude information

In [6]:
#Get data lat/long data from csv
coordinates_df = pd.read_csv("Geospatial_Coordinates.csv")

#rename columns and set the index to be Postcode
coordinates_df.columns = ["Postcode", "Latitude", "Longitude"]
if(coordinates_df.index.name != 'Postcode'):
    coordinates_df = coordinates_df.set_index('Postcode')
    
coordinates_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


### Join latitude and longitude dataframe with neighborhood dataframe

In [7]:
neighbor = df_toronto.join(coordinates_df)
neighbor.head()


,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


### The geograpical coordinate of Toronto, Canada are 43.7170226, -79.4197830350134.

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.7170226, -79.4197830350134], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
scarborough_data.shape

(17, 4)


## The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.

In [15]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[43.773077, -79.257774], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [16]:
CLIENT_ID = 'TY42UREVQEDLYLVBJPLJFQKIOKJIDR5N3Q1OY3ANO0BGGU3M' # your Foursquare ID
CLIENT_SECRET = 'WUQP5MGQ0VVPN2SELT3UYNDLYQK4LFSM3OGANGV0G02G1EZP'
VERSION = '20180605' # Foursquare API versi

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TY42UREVQEDLYLVBJPLJFQKIOKJIDR5N3Q1OY3ANO0BGGU3M
CLIENT_SECRET:WUQP5MGQ0VVPN2SELT3UYNDLYQK4LFSM3OGANGV0G02G1EZP


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [18]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 90 venues in 17 neighborhoods.


In [19]:
print(scarborough_venues.shape)
scarborough_venues.head()

(90, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [20]:
#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",11,11,11,11,11,11
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [21]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

#print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))


There are 80 distinct venues in 53 categories.


In [22]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Home Service,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.00,0.000000,0.000000,0.2,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.2,0.200000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.0,0.000000,0.25,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.250000,0.0,0.000000,0.000000
2,Cedarbrae,0.000000,0.111111,0.000000,0.111111,0.111111,0.0,0.000000,0.0,0.0,0.00,0.111111,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.111111,0.000000,0.111111,0.00,0.000000,0.111111,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.0,0.111111,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.000000,0.000000,0.090909,0.0,0.000000,0.0,0.0,0.00,0.000000,0.090909,0.0,0.000000,0.00,0.0,0.000000,0.000,0.090909,0.090909,0.000000,0.090909,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.090909,0.00,0.0,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.090909,0.0,0.000000,0.090909,0.181818,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0,0.090909,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.000,0.0,0.000,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.166667,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.333333,0.000,0.000000,0.00,0.0,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.1,0.000000,0.100,0.000000,0.00,0.0,0.000000,0.000,0.1,0.000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.125000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.125,0.000000,0.00,0.0,0.000000,0.125,0.0,0.125,0.000000,0.000000,0.000000,0.0,0.000000,0.

#### Each neighborhood with the top 5 venues

In [24]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink   0.2
1  Latin American Restaurant   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4             Clothing Store   0.2
5        American Restaurant   0.0
6                       Park   0.0
7         Italian Restaurant   0.0
8          Korean Restaurant   0.0
9             Medical Center   0.0


----Birch Cliff, Cliffside West----
                       venue  freq
0            College Stadium  0.25
1               Skating Rink  0.25
2      General Entertainment  0.25
3                       Café  0.25
4                       Park  0.00
5         Italian Restaurant  0.00
6          Korean Restaurant  0.00
7  Latin American Restaurant  0.00
8                     Lounge  0.00
9             Medical Center  0.00


----Cedarbrae----
                  venue  freq
0       Thai Restaurant  0.11
1                Bakery  0.11
2                  Bank  0.11
3   Fried Chicken Joint  0.11
4   

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top ten venues for each neighborhood

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Coffee Shop,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Gaming Cafe,Fried Chicken Joint
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Hakka Restaurant,Department Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Italian Restaurant,Gas Station,Thai Restaurant,Chinese Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Noodle House
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Skating Rink,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Gaming Cafe,Chinese Restaurant,Bank,Bar,General Entertainment,Athletics & Sports,Gas Station
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Intersection,Park,Soccer Field,Ice Cream Shop,Metro Station,Bus Station,Department Store,Discount Store
7,"Guildwood, Morningside, West Hill",Mexican Restaurant,Electronics Store,Medical Center,Bank,Restaurant,Rental Car Location,Breakfast Spot,Intersection,College Stadium,Department Store
8,"Kennedy Park, Ionview, East Birchmount Park",Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Hobby Shop,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Gaming Cafe
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store


In [27]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                               Home Service
2nd Most Common Venue                                        Bar
3rd Most Common Venue                                Coffee Shop
4th Most Common Venue                              Grocery Store
5th Most Common Venue                      General Entertainment
6th Most Common Venue                                Gas Station
7th Most Common Venue                                Gaming Cafe
8th Most Common Venue                        Fried Chicken Joint
9th Most Common Venue                       Fast Food Restaurant
10th Most Common Venue                         Electronics Store
Name: 11, dtype: object

In [28]:
# set number of clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0], dtype=int32)

Lets create a new dataframe that includes the cluster as well as the top ten venues for each neighborhood

In [29]:
#Note that the neighborhood Upper Rouge does not have any venues, so I will drop from dataset
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
#df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Home Service,Bar,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Mexican Restaurant,Electronics Store,Medical Center,Bank,Restaurant,Rental Car Location,Breakfast Spot,Intersection,College Stadium,Department Store
3,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,Cedarbrae,43.773136,-79.239476,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Hakka Restaurant,Department Store


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [31]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [32]:
scarborough_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store
1,"Rouge Hill, Port Union, Highland Creek",0,Home Service,Bar,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,"Guildwood, Morningside, West Hill",0,Mexican Restaurant,Electronics Store,Medical Center,Bank,Restaurant,Rental Car Location,Breakfast Spot,Intersection,College Stadium,Department Store
3,Woburn,0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Cedarbrae,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Hakka Restaurant,Department Store
5,Scarborough Village,0,Playground,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,"Kennedy Park, Ionview, East Birchmount Park",0,Coffee Shop,Discount Store,Department Store,Chinese Restaurant,Hobby Shop,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Gaming Cafe
7,"Golden Mile, Clairlea, Oakridge",0,Bakery,Bus Line,Intersection,Park,Soccer Field,Ice Cream Shop,Metro Station,Bus Station,Department Store,Discount Store
8,"Cliffside, Cliffcrest, Scarborough Village West",0,American Restaurant,Motel,Skating Rink,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
11,"Wexford, Maryvale",0,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Bakery,Breakfast Spot,Vietnamese Restaurant,Fast Food Restaurant,Department Store,Discount Store


In [33]:
scarborough_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park, Wexford Heights, Scarborough Town...",1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Gaming Cafe,Chinese Restaurant,Bank,Bar,General Entertainment,Athletics & Sports,Gas Station
12,Agincourt,1,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Coffee Shop,General Entertainment,Gas Station,Gaming Cafe,Fried Chicken Joint


In [34]:
scarborough_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Birch Cliff, Cliffside West",2,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Gaming Cafe,Fried Chicken Joint


## Explore New York City Neighborhoods

New York City has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

In [35]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [36]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [37]:
neighborhoods_data = newyork_data['features']

In [38]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Transform the data into a pandas dataframe

In [39]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [40]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [41]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [42]:
neighborhoods.shape

(306, 4)

In [43]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


The geograpical coordinate of New York City are 40.7308619, -73.9871558.

In [44]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[40.7308619, -73.9871558], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Lets simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Queens neighborhood data.

In [45]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138



The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.

In [47]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[40.6524927, -73.7914214158161], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

#### Lets explore the Long Island City neighborhood in Queens, NY

In [48]:
queens_data.loc[10, 'Neighborhood']

'Long Island City'

In [49]:
#Long Island City Latitude and Longitude values

neighborhood_latitude = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


### Top 100 venues in Long Island City neighborhood within a radius of 500 meters

In [50]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=TY42UREVQEDLYLVBJPLJFQKIOKJIDR5N3Q1OY3ANO0BGGU3M&client_secret=WUQP5MGQ0VVPN2SELT3UYNDLYQK4LFSM3OGANGV0G02G1EZP&v=20180605&ll=40.75021734610528,-73.93920223915505&radius=500&limit=100'

In [51]:
#Send the GET request
results = requests.get(url).json()

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Chip,Dessert Shop,40.750069,-73.940831
1,Etto Espresso Bar,Coffee Shop,40.748703,-73.940689
2,Hilton Garden Inn,Hotel,40.750216,-73.936886
3,Dutch Kills,Cocktail Bar,40.747830,-73.940108
4,Baker House Market,Convenience Store,40.752137,-73.939235


In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


70 venues were returned by Foursquare.


## Analyze Each Neighborhood in Queens

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [56]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude'])

Found 2117 venues in 81 neighborhoods.


In [57]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop


In [58]:
print(queens_venues.shape)
queens_venues.tail()

(2117, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2112,Queensbridge,40.756091,-73.945631,Queensbridge Basketball Courts,40.755060,-73.949103,Basketball Court
2113,Queensbridge,40.756091,-73.945631,The Ravel Hotel Gym,40.753787,-73.948815,Athletics & Sports
2114,Queensbridge,40.756091,-73.945631,Profundo Pool Club,40.753719,-73.948878,Hotel Pool
2115,Queensbridge,40.756091,-73.945631,Estate Garden And Grill,40.753700,-73.948841,Beer Garden
2116,Queensbridge,40.756091,-73.945631,Track 114,40.753008,-73.947833,Platform


In [59]:
#Venues per Neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,21,21,21,21,21,21
Astoria,97,97,97,97,97,97
Astoria Heights,12,12,12,12,12,12
Auburndale,20,20,20,20,20,20
Bay Terrace,39,39,39,39,39,39
Bayside,73,73,73,73,73,73
Bayswater,2,2,2,2,2,2
Beechhurst,14,14,14,14,14,14
Bellaire,12,12,12,12,12,12


In [60]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))


There are 1746 distinct venues in 269 categories.


In [61]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = queens_onehot['Neighborhood']
queens_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
queens_onehot.insert(0, 'Neighborhood', neighbor)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group by Neighborhood, and exame the frequency of the occurrence of venue¶

In [62]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,


Each Neighborhood with the top 5 venues

In [63]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
             venue  freq
0        Surf Spot  0.19
1    Metro Station  0.10
2   Sandwich Place  0.10
3  Thai Restaurant  0.05
4      Pizza Place  0.05


----Astoria----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1                        Bar  0.06
2           Greek Restaurant  0.05
3   Mediterranean Restaurant  0.04
4                 Hookah Bar  0.04


----Astoria Heights----
          venue  freq
0  Burger Joint  0.08
1    Playground  0.08
2  Gourmet Shop  0.08
3          Food  0.08
4        Bakery  0.08


----Auburndale----
                  venue  freq
0    Italian Restaurant  0.10
1     Mobile Phone Shop  0.05
2    Miscellaneous Shop  0.05
3  Fast Food Restaurant  0.05
4        Gymnastics Gym  0.05


----Bay Terrace----
               venue  freq
0     Clothing Store  0.10
1         Kids Store  0.05
2  Mobile Phone Shop  0.05
3     Lingerie Store  0.05
4         Shoe Store  0.05


----Bayside----
                 venue  freq
0               

In [64]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Top venues for each neighborhood

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Sandwich Place,Coffee Shop,Burrito Place,Bus Stop,Café,Board Shop,Bed & Breakfast,Beach
1,Astoria,Middle Eastern Restaurant,Bar,Greek Restaurant,Mediterranean Restaurant,Hookah Bar,Pizza Place,Deli / Bodega,Indian Restaurant,Seafood Restaurant,Bakery
2,Astoria Heights,Plaza,Italian Restaurant,Burger Joint,Bowling Alley,Bus Station,Supermarket,Gourmet Shop,Bakery,Food,Hostel
3,Auburndale,Italian Restaurant,Mattress Store,Gymnastics Gym,Supermarket,Fast Food Restaurant,Noodle House,Toy / Game Store,Sushi Restaurant,Athletics & Sports,Discount Store
4,Bay Terrace,Clothing Store,Women's Store,Lingerie Store,Shoe Store,Mobile Phone Shop,Kids Store,Donut Shop,American Restaurant,Cosmetics Shop,Coffee Shop
5,Bayside,Bar,American Restaurant,Indian Restaurant,Spa,Sushi Restaurant,Pizza Place,Ice Cream Shop,Chinese Restaurant,Greek Restaurant,Mexican Restaurant
6,Bayswater,Construction & Landscaping,Playground,Yoga Studio,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market
7,Beechhurst,Chinese Restaurant,Deli / Bodega,Italian Restaurant,Boutique,Supermarket,Spa,Gym,Park,Donut Shop,Optical Shop
8,Bellaire,Breakfast Spot,Gym,Convenience Store,Coffee Shop,Chinese Restaurant,Moving Target,Bus Stop,Bus Station,Greek Restaurant,Deli / Bodega
9,Belle Harbor,Beach,Spa,Pub,Deli / Bodega,Italian Restaurant,Bakery,Bagel Shop,Boutique,Donut Shop,Pharmacy


In [66]:
neighborhoods_venues_sorted.iloc[47,]

Neighborhood                  Long Island City
1st Most Common Venue                    Hotel
2nd Most Common Venue              Coffee Shop
3rd Most Common Venue                      Bar
4th Most Common Venue              Pizza Place
5th Most Common Venue       Mexican Restaurant
6th Most Common Venue                     Café
7th Most Common Venue               Donut Shop
8th Most Common Venue              Supermarket
9th Most Common Venue     Gym / Fitness Center
10th Most Common Venue           Deli / Bodega
Name: 47, dtype: object

### Cluster the Queens Borough using K-Means

In [67]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [68]:
queens_merged = queens_data

# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,0,Middle Eastern Restaurant,Bar,Greek Restaurant,Mediterranean Restaurant,Hookah Bar,Pizza Place,Deli / Bodega,Indian Restaurant,Seafood Restaurant,Bakery
1,Queens,Woodside,40.746349,-73.901842,0,Grocery Store,Bakery,Filipino Restaurant,Latin American Restaurant,Thai Restaurant,Pizza Place,American Restaurant,Donut Shop,Pub,Bar
2,Queens,Jackson Heights,40.751981,-73.882821,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Mobile Phone Shop,Bakery,Grocery Store,Mexican Restaurant,Thai Restaurant,Empanada Restaurant,Spanish Restaurant
3,Queens,Elmhurst,40.744049,-73.881656,0,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Ice Cream Shop,Sushi Restaurant,Bar,Bakery
4,Queens,Howard Beach,40.654225,-73.838138,0,Italian Restaurant,Bagel Shop,Deli / Bodega,Clothing Store,Spa,Sandwich Place,Fast Food Restaurant,Pharmacy,Donut Shop,Chinese Restaurant


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [69]:
queens_cluster_0 = queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_1 = queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_2 = queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_3 = queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_4 = queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

In [70]:
queens_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,0,Middle Eastern Restaurant,Bar,Greek Restaurant,Mediterranean Restaurant,Hookah Bar,Pizza Place,Deli / Bodega,Indian Restaurant,Seafood Restaurant,Bakery
1,Woodside,0,Grocery Store,Bakery,Filipino Restaurant,Latin American Restaurant,Thai Restaurant,Pizza Place,American Restaurant,Donut Shop,Pub,Bar
2,Jackson Heights,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Mobile Phone Shop,Bakery,Grocery Store,Mexican Restaurant,Thai Restaurant,Empanada Restaurant,Spanish Restaurant
3,Elmhurst,0,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Ice Cream Shop,Sushi Restaurant,Bar,Bakery
4,Howard Beach,0,Italian Restaurant,Bagel Shop,Deli / Bodega,Clothing Store,Spa,Sandwich Place,Fast Food Restaurant,Pharmacy,Donut Shop,Chinese Restaurant
5,Corona,0,Mexican Restaurant,Supermarket,Ice Cream Shop,Bakery,Italian Restaurant,Empanada Restaurant,Donut Shop,Park,Pizza Place,Deli / Bodega
7,Kew Gardens,0,Chinese Restaurant,Cosmetics Shop,Bar,Park,Bank,Donut Shop,Juice Bar,Pet Store,Pizza Place,Indian Restaurant
8,Richmond Hill,0,Pizza Place,Lounge,Latin American Restaurant,Bank,Sandwich Place,Park,Moving Target,Supermarket,Clothing Store,Donut Shop
9,Flushing,0,Hotpot Restaurant,Bubble Tea Shop,Chinese Restaurant,Korean Restaurant,Bakery,Karaoke Bar,Gym / Fitness Center,Gym,Construction & Landscaping,BBQ Joint
10,Long Island City,0,Hotel,Coffee Shop,Bar,Pizza Place,Mexican Restaurant,Café,Donut Shop,Supermarket,Gym / Fitness Center,Deli / Bodega


In [71]:
queens_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,1,Deli / Bodega,Ice Cream Shop,Automotive Shop,Pizza Place,Park,Donut Shop,South American Restaurant,Spa,Basketball Court,Grocery Store


In [72]:
queens_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Glendale,2,Brewery,Food & Drink Shop,Pizza Place,Arts & Crafts Store,Deli / Bodega,Food Court,Food,Flower Shop,Flea Market,Empanada Restaurant


In [73]:
queens_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Forest Hills,3,Gym / Fitness Center,Gym,Yoga Studio,Pizza Place,Park,Convenience Store,Thai Restaurant,Asian Restaurant,Pharmacy,Video Game Store


In [74]:
queens_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Arverne,4,Surf Spot,Metro Station,Sandwich Place,Coffee Shop,Burrito Place,Bus Stop,Café,Board Shop,Bed & Breakfast,Beach
